# Indice CHF

## Introducción

En este trabajo, se aborda la creación de un índice esencial para comprender la climatología de huracanes en las regiones de Costa Rica y Nicaragua: el CHF (Costal Hurricane Frequency), implementado en el articulo de Karthik Balaguru, "Increased U.S. coastal hurricane risk under climate change". En este estudio se emplea una base de datos de HURDAT (Hurricane Database) para obtener una estimación del CHF para las regiones previamente mencionadas. Es importante destacar que, si bien se utiliza la técnica CHF como punto de partida, se han realizado ajustes para adaptarla a las características que tiene la base de datos del HURDAT.

El CHF se define como la cantidad de ubicaciones de trayectorias de huracanes que cruzan sobre tierra durante un periodo de seis horas por grado cuadrado por año. Para su cálculo, se consideran todas las ubicaciones de tormentas cuya intensidad supera un umbral específico. El cual tiene la siguiente ecuación:

$$
CHF = n\frac{l}{TimeStep*v}
$$

En donde, $$
n: \textrm{es el número de trayectorias de huracanes que pasan a través de un grado cuadrado}
$$ $$
l: \textrm{la longitud de la cuadrícula}
$$
$$
v: \textrm{la velocidad de traslación}
$$
$$
TimeStep: \textrm{es el paso de tiempo}
$$

## Metodología 

Se utilizara Python para el análisis de este cuaderno, en donde se utilizaron las siguientes librerias

### Librerias

-   **NumPy:** Libreria para realizar cálculos numéricos en Python. Es ampliamente utilizada para operaciones matemáticas y de manipulación de datos.

-   **Pandas:** Pandas es una biblioteca que proporciona estructuras de datos y herramientas para el análisis de datos. Es especialmente útil para la manipulación y exploración de datos tabulares.

-   **GeoPandas:** GeoPandas es una extensión de Pandas que agrega capacidades geoespaciales. Permite trabajar con datos geoespaciales, como GeoDataFrames, y realizar análisis espaciales.

-   **OS:** El módulo `os` proporciona funciones para interactuar con el sistema operativo, como la manipulación de rutas de archivos y directorios.

-   **Shapely:** Shapely es una biblioteca que se utiliza para el procesamiento y análisis de geometría plana. Permite trabajar con objetos geométricos como puntos, líneas y polígonos.

-   **pyogrio:** pyogrio es una biblioteca que se utiliza para leer y escribir datos geoespaciales en formatos como GeoJSON y Shapefile. Facilita la manipulación de datos geoespaciales.

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os 
import shapely
from pyogrio import read_dataframe
from shapely.geometry import Polygon, Point

Para la creación del indice del CHF, tal como se menciono en la introducción se utilizaron los datos del HURDAT. Dentro de estos datos hay dos bases la que contiene un intervalo de tiempo de los huracanes de 6 horas y otro de 24 horas. Cada uno teniendo su ventaja y desventaja, algunas de estas son:

**Base de Datos de 6 Horas:**

*Ventajas:*
- Mayor resolución temporal: Registra datos de los huracanes cada 6 horas, lo que proporciona una visión detallada de la evolución de los fenómenos.
- Captura cambios rápidos: Permite identificar cambios súbitos en la intensidad y trayectoria de los huracanes.
- Alineación con la teoría de Balaguru: Esta base se ajusta mejor a la teoría que utiliza un intervalo de 6 horas para evaluar el desplazamiento de los huracanes.

*Desventajas:*
- Mayor volumen de datos: La mayor cantidad de puntos de datos puede requerir más recursos de procesamiento y almacenamiento.
- Posibles datos redundantes: La alta frecuencia de registros podría redundar en información similar en períodos cortos de tiempo.

**Base de Datos de 24 Horas:**

*Ventajas:*
- Menor volumen de datos: Contiene menos puntos de datos, lo que puede simplificar la gestión de datos y el análisis.
- Posiblemente menos ruido: Al tener un intervalo de tiempo más amplio entre registros, es posible que haya menos variabilidad aleatoria en los datos.

*Desventajas:*
- Menor resolución temporal: Registra datos de los huracanes cada 24 horas, lo que podría no capturar cambios rápidos en la intensidad y trayectoria.
- Pérdida de detalles: La menor frecuencia de registros podría resultar en la pérdida de información importante sobre la evolución de los huracanes.

Por lo cual se opto por hacer el análisis a ambas bases de datos.

## Análsis 

### Base de datos 

Seguidamente, se procede a importar las bases de datos del HURDAT (Hurricane Database), en conjunto con los archivos de shapefiles que representan los límites geográficos de los países de Costa Rica y Nicaragua.

In [2]:
hd_24 = pd.read_csv('Datos/hurdat2daily.csv')
hd_24 =  gpd.GeoDataFrame(hd_24,
                                geometry=gpd.points_from_xy(hd_24.Lon, hd_24.Lat), 
                                crs="EPSG:4326") #Base del Hurdat con formato geoespacial

In [1]:
#hd_6 = pd.read_csv('Datos/hurdat2.txt',sep=" ")

In [3]:
#Importamos los paises de CR y NIC; y los concatenamos

cr = read_dataframe("Datos/Shapefiles/CR/CR.shp")
nic = read_dataframe("Datos/Shapefiles/NIC/NIC.shp")

cr_nic = pd.concat([cr,nic]) #Concatenamos ambas bases
cr_nic["Pais"] = ["CR","NIC"] #Designamos cual geometría le pertenece a que país

Debido al formato en el cuál se paso la información por el investigador Dr. Hugo Hidalgo no se puede importar por lo cual por el momento solo se trabajara con la del Hurdat de 24 horas.

### Filtración y Manipulación

Para esta sub seccion se filtrara la base de datos para tener un mejor entendimiento de la data, además se filtraran dos bases una que contiene todos los huracanes desde 1891 hasta 2020, mientras que la otra tendrá información más reciente, es decir desde el 2016. 

In [4]:
hd_24[["year","month","day"]] = hd_24["dat"].str.split('-',expand=True) #Separamos la fecha

hd_24.drop(columns= 'dat', inplace= True) #Eliminamos la columna que contiene la fecha agrupada

hd_24 = hd_24.astype({'year':'int'})

In [5]:
#Todos los años

#minx, miny, maxx, maxy = cr_nic.total_bounds #Limites de los paises de Nicaragua y Costa Rica
#cr_nic = hd_24[((hd_24["Lon"]<= maxx)&
                            #(hd_24["Lon"]>= minx))&
                            #((hd_24["Lat"]<= maxy)&
                            #(hd_24["Lat"]>= miny))] #Delimitación de la zona

In [6]:
# Dividimos la base del Hurdat para year>2016 y year>1891

hd_24_2016 = hd_24[hd_24['year']>=2016]

In [7]:
#Años de 2016 en adelante

minx, miny, maxx, maxy = cr_nic.total_bounds #Limites de los paises de Nicaragua y Costa Rica
cr_nic_hur = hd_24_2016[((hd_24_2016["Lon"]<= maxx)&
                            (hd_24_2016["Lon"]>= minx))&
                            ((hd_24_2016["Lat"]<= maxy)&
                            (hd_24_2016["Lat"]>= miny))] #Delimitación de la zona

## Creación del Indicador CHF

En esta sección se trabajara con el indicadoPrimero se empieza creando las cuadriculas que sugiere el indice de Balagurú.

In [8]:
grado_cuad = 1 #Grado cuadrado = metro cuadrado

x_coords = np.arange(minx, maxx, grado_cuad)
y_coords = np.arange(miny, maxy, grado_cuad)

celdas = [] #Lista donde guardaremos las coordenadas de las celdas

for x in x_coords:
    for y in y_coords:
        poligono = Polygon([(x, y), 
        (x + grado_cuad, y), 
        (x + grado_cuad, y + grado_cuad), 
        (x, y + grado_cuad)])
        celdas.append(poligono)
        
celdas[:5] #Mostramos solo los primeros 5 resultados

[<POLYGON ((-87.7 5.5, -86.7 5.5, -86.7 6.5, -87.7 6.5, -87.7 5.5))>,
 <POLYGON ((-87.7 6.5, -86.7 6.5, -86.7 7.5, -87.7 7.5, -87.7 6.5))>,
 <POLYGON ((-87.7 7.5, -86.7 7.5, -86.7 8.5, -87.7 8.5, -87.7 7.5))>,
 <POLYGON ((-87.7 8.5, -86.7 8.5, -86.7 9.5, -87.7 9.5, -87.7 8.5))>,
 <POLYGON ((-87.7 9.5, -86.7 9.5, -86.7 10.5, -87.7 10.5, -87.7 9.5))>]

In [10]:
grid_gdf = gpd.GeoDataFrame({'geometry': celdas}, crs=cr_nic.crs)

# Calcular la frecuencia de huracanes en cada cuadrícula
grid_gdf['frecuencia_h'] = grid_gdf['geometry'].apply(lambda geom: len(cr_nic_hur[cr_nic_hur.within(geom)]))

# Calcular el promedio de velocidad de huracanes en cada cuadrícula
grid_gdf['velocidad_promedio'] = grid_gdf['geometry'].apply(lambda geom: cr_nic_hur[cr_nic_hur.within(geom)]['MaxW'].mean())

# Sumar el promedio de velocidad a la frecuencia en un nuevo campo
grid_gdf['Indicador_CHF'] = grid_gdf['frecuencia_h']/(grid_gdf['velocidad_promedio']+24)

grid_gdf[grid_gdf["frecuencia_h"]>0].head()

,geometry,frecuencia_h,velocidad_promedio,Indicador_CHF
17,"POLYGON ((-86.69097 12.49903, -85.69097 12.499...",1,52.00,0.013158
28,"POLYGON ((-85.69097 13.49903, -84.69097 13.499...",2,58.75,0.024169
35,"POLYGON ((-84.69097 10.49903, -83.69097 10.499...",1,85.00,0.009174
39,"POLYGON ((-84.69097 14.49903, -83.69097 14.499...",1,36.20,0.016611
45,"POLYGON ((-83.69097 10.49903, -82.69097 10.499...",1,43.00,0.014925


In [11]:
documento = 'Indice_CHF.geojson'
with open(documento , 'w') as file:
    file.write(grid_gdf.to_json())

In [13]:
documento = 'Huracanes_CR_NIC.geojson'
with open(documento , 'w') as file:
    file.write(cr_nic_hur.to_json())

In [14]:
documento = 'CR_NIC.geojson'
with open(documento , 'w') as file:
    file.write(cr_nic.to_json())